<div style="text-align: center;background-color: gold; padding: 25px; border-radius: 5px; margin-bottom: 10px;">
    <h2> MEAT QUANTITY PREDICTION USING MACHINE LEARNING AND Statistical Approach.</h2>
</div>

* Main Idea : `Robi Datathon 3.0 Problem 3` 😊✨
* Coding    : `Tanbeer Jubaer` 🌟 and `Minhazul Islam`  and  `Shaikh Hossain Elin` 🏆

# Dataset Description
For this competition,

The MAGIC_KEY id is the unique identifier and the rest columns are the features.

“problem 3.csv” contains the Magic Keys of customers who purchased at least one box of milk and/or meat in the first 15 days of March-2019. You need to predict what quantity of meat were be purchased by them in this period. Follow "sample submission 3.csv" as the submission template.

## Files

    boxes.csv - contains box informations
    purchase.csv - contains a product purchase info
    problem 2.csv - the test set
    sample submission 2.csv - a sample submission file in the correct format

<div style="text-align: center;">
    <h2> 1. Import Necessary Libraries</h2>
</div>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

%matplotlib inline

from tqdm import tqdm
import string
import statistics
import re
import joblib
import random
import statistics
from collections import Counter
from scipy import stats
import os

# import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import zipfile
from wordcloud import WordCloud

## Data Import

In [ ]:
box = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-3/boxes.csv')
box

In [ ]:
prb3 = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-3/problem 3.csv')
prb3

In [ ]:
purchase = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-3/purchase.csv')
purchase

In [ ]:
sub = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-3/sample submission 3.csv')
sub

In [ ]:
purchase.MAGIC_KEY.nunique()/len(purchase)

<div style="text-align: center; background-color: orange; padding: 20px;">
    <h3>Exploratory Data Analysis</h3>
</div>


In [ ]:
l1=[]
l2=[]
for i in prb3['MAGIC_KEY']:
    l1.append(i)
for i in purchase['MAGIC_KEY']:
    l2.append(i)
    
print(len(l1),len(l2))
l1=set(l1)
l2=set(l2)
l3=l1.intersection(l2)
print(len(l3))

In [ ]:
box.rename(columns=lambda x: x.lower(), inplace=True)
purchase.rename(columns=lambda x: x.lower(), inplace=True)

In [ ]:
purchase.isna().sum()
purchase.dropna(inplace=True)

In [ ]:
print(box.shape)
print()
print(purchase.shape)

In [ ]:
print(box.dtypes)
print()
print(purchase.dtypes)

In [ ]:
purchase['box_id'] = purchase['box_id'].astype(int)

In [ ]:
merged_df = pd.merge(box, purchase, on='box_id', how='left')
merged_df.head()

In [ ]:
cli = prb3['MAGIC_KEY'].tolist()

cli[:4], len(cli)

In [ ]:
fmerged = merged_df[merged_df['magic_key'].isin(cli)].copy()

fmerged.head()

In [ ]:
fmerged.magic_key.nunique()

## Convert to Categorical Values

In [ ]:
fmerged['nquality'] = fmerged['quality'].astype('category').cat.codes
fmerged['ndelivery'] = fmerged['delivery_option'].astype('category').cat.codes

category_mapping = dict(enumerate(fmerged['quality'].astype('category').cat.categories))
category_mapping2 = dict(enumerate(fmerged['delivery_option'].astype('category').cat.categories))

print(category_mapping)

print(category_mapping2)

In [ ]:
fmerged.drop(['quality', 'delivery_option'], axis=1,inplace=True)

In [ ]:
fmerged.head(2)

In [ ]:
fmerged['date'] = pd.to_datetime(fmerged['purchase_date'], format='%d/%m/%Y')

fmerged['date'] = fmerged['date'].dt.date
most_previous_date = fmerged['date'].min()
most_recent_date = fmerged['date'].max()

print("Most previous date:", most_previous_date)
print("Most recent date:", most_recent_date)

In [ ]:
fmerged.drop(['purchase_date'], axis=1, inplace=True)
fmerged.head(2)

In [165]:
sorted_df = fmerged.groupby('magic_key').apply(lambda x: x.sort_values('date'), include_groups=True).reset_index(drop=True)

sorted_df.head(3)

/tmp/ipykernel_33/3906745481.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = fmerged.groupby('magic_key').apply(lambda x: x.sort_values('date'), include_groups=True).reset_index(drop=True)


,box_id,milk,meat,unit_price,magic_key,box_count,nquality,ndelivery,date
0,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-02
1,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-09
2,90,10.5,0.0,12.78,2499B9E0C1C,1.0,0,0,2019-02-18


In [166]:
# from tqdm import tqdm

# # Sort the DataFrame by 'date'
# fmerged_sorted = fmerged.sort_values(by='date')

# # Group the sorted DataFrame by 'magic_key'
# grouped_sorted_df = fmerged_sorted.groupby('magic_key')

# # Create an empty list to store the sorted and grouped data
# dfs = []

# # Use tqdm to track the progress of the loop
# for _, group in tqdm(grouped_sorted_df, total=len(grouped_sorted_df), desc="Processing groups"):
#     dfs.append(group)

# # Concatenate the list of DataFrames into a single DataFrame
# sorted_df = pd.concat(dfs, ignore_index=True)

# sorted_df.head(3)


In [167]:
# sorted_df.to_csv('sorted_df.csv', index=False)

In [168]:
ndata = sorted_df[:200].copy()

ndata[:10]

,box_id,milk,meat,unit_price,magic_key,box_count,nquality,ndelivery,date
0,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-02
1,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-09
2,90,10.5,0.0,12.78,2499B9E0C1C,1.0,0,0,2019-02-18
3,230,8.4,0.0,10.14,24D1CF7CA25,1.0,1,0,2019-02-11
4,230,8.4,0.0,10.14,24D1CF7CA25,1.0,1,0,2019-02-21
5,118,8.0,1.3,15.98,24D279F5F06,1.0,0,0,2018-10-03
6,132,9.0,1.8,19.98,27D86DD37C9,1.0,0,0,2018-10-08
7,163,11.0,2.2,23.98,27D86DD37C9,1.0,0,0,2018-11-09
8,163,11.0,2.2,23.98,27D86DD37C9,1.0,0,0,2018-12-22
9,258,8.0,2.2,17.98,28CF0640F8B,1.0,1,0,2018-11-17


## these features didn't worked

In [169]:
def find_std(df):
    df['dis']=df['date'].diff().dt.days
    if df.dis.count()==1:
        return .98
    x=df['dis'].std()
    return x

def find_mean(df):
    df['dis']=df['date'].diff().dt.days
    x=df['dis'].mean()
    return x
    
cur_date = pd.to_datetime('13/03/2019', format='%d/%m/%Y').date()

print(cur_date)

def find_dis(df):
    df=df.reset_index(drop=True)
    index=df.tail(1).index[0]
    date=df['date'][index]
#     print(date)
    diff=cur_date-date
#     print(diff)
    

2019-03-13


In [170]:
ndata.head(2)

,box_id,milk,meat,unit_price,magic_key,box_count,nquality,ndelivery,date
0,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-02
1,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-09


In [171]:
ndata[:20]

,box_id,milk,meat,unit_price,magic_key,box_count,nquality,ndelivery,date
0,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-02
1,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-09
2,90,10.5,0.0,12.78,2499B9E0C1C,1.0,0,0,2019-02-18
3,230,8.4,0.0,10.14,24D1CF7CA25,1.0,1,0,2019-02-11
4,230,8.4,0.0,10.14,24D1CF7CA25,1.0,1,0,2019-02-21
5,118,8.0,1.3,15.98,24D279F5F06,1.0,0,0,2018-10-03
6,132,9.0,1.8,19.98,27D86DD37C9,1.0,0,0,2018-10-08
7,163,11.0,2.2,23.98,27D86DD37C9,1.0,0,0,2018-11-09
8,163,11.0,2.2,23.98,27D86DD37C9,1.0,0,0,2018-12-22
9,258,8.0,2.2,17.98,28CF0640F8B,1.0,1,0,2018-11-17


In [172]:
sorted_df.magic_key.nunique()

5378

In [173]:
for index,row in sorted_df[:1000].iterrows():
    if row['magic_key']=='2499B9E0C1C':
        print(row,'\n')

box_id                 89
milk                 10.0
meat                  0.0
unit_price          12.18
magic_key     2499B9E0C1C
box_count             1.0
nquality                0
ndelivery               0
date           2019-02-02
Name: 0, dtype: object 

box_id                 89
milk                 10.0
meat                  0.0
unit_price          12.18
magic_key     2499B9E0C1C
box_count             1.0
nquality                0
ndelivery               0
date           2019-02-09
Name: 1, dtype: object 

box_id                 90
milk                 10.5
meat                  0.0
unit_price          12.78
magic_key     2499B9E0C1C
box_count             1.0
nquality                0
ndelivery               0
date           2019-02-18
Name: 2, dtype: object 



In [174]:
ndata.head(1)

,box_id,milk,meat,unit_price,magic_key,box_count,nquality,ndelivery,date
0,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-02


<div style="text-align: center; background-color: orange; padding: 20px;">
    <h3>Plotting The Correlation</h3>
</div>


In [1]:
numeric_columns = ['box_id', 'milk','meat','unit_price', 'nquality', 'ndelivery']
df_numeric = sorted_df[numeric_columns]

# Calculate Pearson correlation coefficient
correlation = df_numeric.corr()

print(correlation)

NameError: name 'sorted_df' is not defined

In [ ]:
import seaborn as sns

# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 5))  # Adjust the figure size as needed
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt=".2f")

# Add title and adjust plot settings
plt.title('Correlation Matrix')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()

# Show plot
plt.show()

In [177]:
meat_correlation = correlation['meat']

print(meat_correlation)

box_id       -0.108512
milk         -0.408289
meat          1.000000
unit_price    0.504535
nquality     -0.310403
ndelivery    -0.162967
Name: meat, dtype: float64


In [178]:
sorted_df

,box_id,milk,meat,unit_price,magic_key,box_count,nquality,ndelivery,date
0,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-02
1,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-09
2,90,10.5,0.0,12.78,2499B9E0C1C,1.0,0,0,2019-02-18
3,230,8.4,0.0,10.14,24D1CF7CA25,1.0,1,0,2019-02-11
4,230,8.4,0.0,10.14,24D1CF7CA25,1.0,1,0,2019-02-21
...,...,...,...,...,...,...,...,...,...
15348,166,12.0,2.9,23.98,2E6B1F1DFFA,1.0,0,0,2018-12-02
15349,163,11.0,2.2,23.98,2E6B1F28267,1.0,0,0,2018-11-02
15350,134,10.0,1.8,19.98,2E6B1F28267,1.0,0,0,2018-12-01
15351,162,10.0,2.2,23.98,2E6B1F28267,1.0,0,0,2019-01-01


In [179]:
sorted_df['mk'] = sorted_df['magic_key'].astype('category').cat.codes

magickey_map = dict(enumerate(sorted_df['magic_key'].astype('category').cat.categories))

# magickey_map

In [180]:
# Extract features from datetime (e.g., year, month)
sorted_df['year'] = pd.to_datetime(sorted_df['date']).dt.year
sorted_df['month'] = pd.to_datetime(sorted_df['date']).dt.month
sorted_df['day'] = pd.to_datetime(sorted_df['date']).dt.day

In [181]:
sorted_df.head(2)

,box_id,milk,meat,unit_price,magic_key,box_count,nquality,ndelivery,date,mk,year,month,day
0,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-02,0,2019,2,2
1,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-09,0,2019,2,9



<div style="text-align: center; background-color: orange; padding: 20px;">
    <h3>No Machine Learning Approach</h3>
</div>


In [182]:
sorted_df.head(3)

,box_id,milk,meat,unit_price,magic_key,box_count,nquality,ndelivery,date,mk,year,month,day
0,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-02,0,2019,2,2
1,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-09,0,2019,2,9
2,90,10.5,0.0,12.78,2499B9E0C1C,1.0,0,0,2019-02-18,0,2019,2,18


In [183]:
sorted_df.magic_key.nunique()

5378

In [184]:
print(category_mapping)

print(category_mapping2)

{0: 'Premium', 1: 'Standard'}
{0: 'Delivery from Collection Point', 1: 'Home Delivery - CoD', 2: 'Home Delivery - Digital Payment'}


In [185]:
sorted_df.meat.value_counts().sort_index()

meat
0.0    5863
0.1       6
0.2      12
1.1      21
1.2       7
1.3     467
1.5     231
1.6     264
1.8    1526
2.0     156
2.2    2195
2.3       4
2.4    1165
2.5     524
2.6       1
2.7     124
2.8       2
2.9     686
3.1       5
3.2       1
3.3     483
3.5      53
3.6    1228
3.7       1
3.9       1
4.0     285
4.4      40
4.7       1
5.8       1
Name: count, dtype: int64

In [186]:
key_frequency = sorted_df.groupby('magic_key').size().reset_index(name='freq')

sorted_df = sorted_df.merge(key_frequency, on='magic_key', how='left')

sorted_df

,box_id,milk,meat,unit_price,magic_key,box_count,nquality,ndelivery,date,mk,year,month,day,freq
0,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-02,0,2019,2,2,3
1,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-09,0,2019,2,9,3
2,90,10.5,0.0,12.78,2499B9E0C1C,1.0,0,0,2019-02-18,0,2019,2,18,3
3,230,8.4,0.0,10.14,24D1CF7CA25,1.0,1,0,2019-02-11,1,2019,2,11,2
4,230,8.4,0.0,10.14,24D1CF7CA25,1.0,1,0,2019-02-21,1,2019,2,21,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15348,166,12.0,2.9,23.98,2E6B1F1DFFA,1.0,0,0,2018-12-02,5376,2018,12,2,1
15349,163,11.0,2.2,23.98,2E6B1F28267,1.0,0,0,2018-11-02,5377,2018,11,2,4
15350,134,10.0,1.8,19.98,2E6B1F28267,1.0,0,0,2018-12-01,5377,2018,12,1,4
15351,162,10.0,2.2,23.98,2E6B1F28267,1.0,0,0,2019-01-01,5377,2019,1,1,4


In [187]:
sorted_df['date'][0]

datetime.date(2019, 2, 2)

In [188]:
import datetime

date_15_feb_2019 = datetime.date(2019, 2, 15)
mask = sorted_df['date'] > date_15_feb_2019
sorted_df['f15'] = mask.astype(int)

In [189]:
sorted_df

,box_id,milk,meat,unit_price,magic_key,box_count,nquality,ndelivery,date,mk,year,month,day,freq,f15
0,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-02,0,2019,2,2,3,0
1,89,10.0,0.0,12.18,2499B9E0C1C,1.0,0,0,2019-02-09,0,2019,2,9,3,0
2,90,10.5,0.0,12.78,2499B9E0C1C,1.0,0,0,2019-02-18,0,2019,2,18,3,1
3,230,8.4,0.0,10.14,24D1CF7CA25,1.0,1,0,2019-02-11,1,2019,2,11,2,0
4,230,8.4,0.0,10.14,24D1CF7CA25,1.0,1,0,2019-02-21,1,2019,2,21,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15348,166,12.0,2.9,23.98,2E6B1F1DFFA,1.0,0,0,2018-12-02,5376,2018,12,2,1,0
15349,163,11.0,2.2,23.98,2E6B1F28267,1.0,0,0,2018-11-02,5377,2018,11,2,4,0
15350,134,10.0,1.8,19.98,2E6B1F28267,1.0,0,0,2018-12-01,5377,2018,12,1,4,0
15351,162,10.0,2.2,23.98,2E6B1F28267,1.0,0,0,2019-01-01,5377,2019,1,1,4,0


In [190]:
sorted_df.freq.value_counts().sort_index()

freq
1     2859
2     1598
3     1347
4     1360
5     1190
6      990
7      756
8      584
9      576
10     610
11     396
12     408
13     390
14     378
15     210
16     192
17     136
18     252
19     114
20     100
21     147
22      88
23     115
24     120
25     125
26      78
29      29
31      31
34      68
35      70
36      36
Name: count, dtype: int64

In [191]:
reserve = sorted_df.copy()

In [216]:
ri = np.random.randint(0, 1000)

print('index', ri)

filtered_rows = sorted_df[(sorted_df['magic_key'] == cli[ri])]

# Print the desired columns of the filtered rows
print(filtered_rows[['meat', 'milk', 'date', 'unit_price', 'nquality', 'ndelivery']])


index 339
       meat  milk        date  unit_price  nquality  ndelivery
13451   3.6  10.0  2019-02-26        15.0         0          0
13452   3.6  10.0  2019-02-27        15.0         0          0
13453   3.6  10.0  2019-02-28        15.0         0          0
13454   3.6  10.0  2019-02-28        15.0         0          0


In [195]:
idx_to_keep = sorted_df.groupby('magic_key')['date'].idxmax()

# Keep only the rows with the indices found above
filtered_df = sorted_df.loc[idx_to_keep]
filtered_df = filtered_df[filtered_df['magic_key'].isin(cli)].copy()

filtered_df.head()

,box_id,milk,meat,unit_price,magic_key,box_count,nquality,ndelivery,date,mk,year,month,day,freq,f15
2,90,10.5,0.0,12.78,2499B9E0C1C,1.0,0,0,2019-02-18,0,2019,2,18,3,1
4,230,8.4,0.0,10.14,24D1CF7CA25,1.0,1,0,2019-02-21,1,2019,2,21,2,1
5,118,8.0,1.3,15.98,24D279F5F06,1.0,0,0,2018-10-03,2,2018,10,3,1,0
8,163,11.0,2.2,23.98,27D86DD37C9,1.0,0,0,2018-12-22,3,2018,12,22,3,0
12,238,10.7,0.0,12.78,28CF0640F8B,1.0,1,0,2019-02-25,4,2019,2,25,4,1


In [196]:
sorted_df.nquality.value_counts()

nquality
0    12114
1     3239
Name: count, dtype: int64

<div style="text-align: center; background-color: orange; padding: 20px;">
    <h3>The Main Trick Part is Here Do All The Experimentation in Here </h3>
</div>

### Here We observed that the growth of meat sale per mounth is 11-14% so we tried to apply this growth in them based on some additional criterias like 
1. how frequent the customer buys.
2. Did he purchased on the last 15 days of February.

In [229]:
rli2 = []
for i in tqdm(range(len(cli[1828:1830]))):
    temp = filtered_df[filtered_df['magic_key'] == cli[i]][['meat','freq', 'f15']].reset_index(drop=True)
    print(temp)
    meat = temp['meat'].iloc[0] 
    freq = temp['freq'].iloc[0] 
    f15 = temp['f15'].iloc[0] 
    if(cli[i] == '290D33249B7'):
        rli2.append(0)
    elif freq > 4:
#         updated_meat = meat + 0.11 * meat  
        rli2.append(meat)
    else:
        updated_meat = meat + 0.11 * meat  
        rli2.append(updated_meat)

100%|██████████| 2/2 [00:00<00:00, 141.77it/s]

   meat  freq  f15
0   3.6     5    1
   meat  freq  f15
0   2.2     2    0


In [220]:
rli2

[3.6, 2.442, 2.442, 3.6, 2.442, 2.442, 2.442, 3.6, 3.996, 2.442]

In [221]:
np.sum([3.6, 2.2, 2.2]), np.sum(rli2)

(8.0, 29.447999999999997)

In [233]:
rli2 = []
for i in tqdm(range(len(cli))):
    temp = filtered_df[filtered_df['magic_key'] == cli[i]][['meat','freq', 'f15']]
#     print(temp)
    if(cli[i] == '290D33249B7'):
        rli2.append(0)
    else:
        meat = temp['meat'].iloc[0]
        freq = temp['freq'].iloc[0] 
        f15 = temp['f15'].iloc[0] 
        if freq > 4:
#             updated_meat = meat + 0.11 * meat  
            rli2.append(meat)
        else:
            updated_meat = meat + 0.11 * meat  
            rli2.append(updated_meat)

100%|██████████| 5379/5379 [00:13<00:00, 404.69it/s]


In [234]:
rli2[:10]

[3.6, 2.442, 2.442, 3.6, 2.442, 2.442, 2.442, 3.6, 3.996, 2.442]

## There was no data for this magic key = 290D33249B7 so sometimes we filled it with the mean or a random value

In [236]:
prb3['MEAT'] = rli2
prb3.at[1829, 'MEAT'] = 2.0
prb3['MEAT'][1829]

2.0

<div style="text-align: center; background-color: orange; padding: 20px;">
    <h3>File Submission </h3>
</div>


In [237]:
prb3

,MAGIC_KEY,MEAT
0,2BCFE9C06A7,3.600
1,2C2A872B5A2,2.442
2,2C6A897671B,2.442
3,2C6F1287F53,3.600
4,2C658198CC9,2.442
...,...,...
5374,2C0804EFE49,2.775
5375,2C080B48630,0.000
5376,2C08243C58E,2.664
5377,2C082C78575,1.998


In [239]:
prb3.to_csv('submission810.csv', index=False)

<div style="text-align: center; background-color: orange; padding: 20px;">
    <h3>  ML Based Appproach </h3>
</div>


In [145]:
from sklearn.preprocessing import StandardScaler  
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error  

data = sorted_df[:1200000].copy()


features = sorted_df.drop(['meat', 'magic_key', 'date', 'box_count', 'ndelivery'], axis=1)
target = sorted_df['meat']

In [146]:
features.head(2)

,box_id,milk,unit_price,nquality,mk,year,month,day,freq
0,89,10.0,12.18,0,0,2019,2,2,3
1,89,10.0,12.18,0,0,2019,2,9,3


In [147]:
target.head(2)

0    0.0
1    0.0
Name: meat, dtype: float64

In [148]:
# Feature scaling (optional, but recommended for some models)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)



# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target, test_size=0.1)
X_train

array([[-1.34469777, -1.70272794, -0.42110384, ...,  0.88384289,
        -0.21053052, -0.90796712],
       [-0.05435075, -0.0931598 ,  0.06586548, ..., -1.01353707,
         0.69763512, -0.90796712],
       [ 0.20371866,  0.71162427,  1.03016117, ...,  1.09466288,
         1.03819723, -0.61052294],
       ...,
       [ 0.13580565,  0.10803622,  1.03016117, ...,  0.88384289,
        -1.23221685, -0.75924503],
       [-0.89647196,  0.30923224, -0.17038696, ..., -0.80271708,
         1.15171794, -0.31307875],
       [ 0.54328366,  0.30923224,  1.99445686, ..., -1.01353707,
         0.47059371, -0.90796712]])

In [149]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)


len(X_train), len(X_test)

(12282, 3071)

In [150]:
X_train.head(3)

,box_id,milk,unit_price,nquality,mk,year,month,day,freq
3761,89,10.0,12.18,0,997,2018,12,2,25
54,230,8.4,10.14,1,33,2019,2,19,1
14679,163,11.0,23.98,0,5205,2018,10,15,7


In [ ]:
X_test.head(2)

In [ ]:
import xgboost as xgb

xgb_reg = xgb.XGBRegressor()

xgb_reg.fit(X_train, y_train)

y_pred_xgb = xgb_reg.predict(X_test)

In [ ]:
# Make predictions and evaluate
if 20>3:
    y_pred = xgb_reg.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)  # Calculate RMSE
    r2 = r2_score(y_test, y_pred)  # Calculate R-squared

    # Print results for the best model configuration
    print(f"RMSE: {rmse:.4f}")
    print(f"R-squared: {r2:.4f}")
    print("-" * 30)  #


In [ ]:
y_test[:5]

In [ ]:
y_pred_xgb[:5]

In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define the CatBoostRegressor model
catboost_reg = CatBoostRegressor()

# Train the model
catboost_reg.fit(X_train, y_train, verbose=0)  # Setting verbose=0 to suppress output during training

# Predict on test data
y_pred_catboost = catboost_reg.predict(X_test)

# Calculate evaluation metrics
rmse_catboost = mean_squared_error(y_test, y_pred_catboost, squared=False)
r2_catboost = r2_score(y_test, y_pred_catboost)

# Print results
print("CatBoost Regressor:")
print(f"RMSE: {rmse_catboost:.4f}")
print(f"R-squared: {r2_catboost:.4f}")


In [ ]:
prb3.head(2)

In [ ]:
magickey_map = {value: key for key, value in magickey_map.items()}

In [ ]:
magickey_map['290D33249B7'] = 100000

In [ ]:
prb3['magic_key'] = [magickey_map[x] for x in prb3.MAGIC_KEY]

In [ ]:
prb3.head(3)

In [ ]:
sorted_df

In [ ]:
idx_to_keep = sorted_df.groupby('magic_key')['date'].idxmax()

# Keep only the rows with the indices found above
filtered_df = sorted_df.loc[idx_to_keep]
filtered_df = filtered_df[filtered_df['magic_key'].isin(cli)].copy()

In [ ]:
filtered_df.shape

In [ ]:
filtered_df

In [ ]:
cli[:3], magickey_map['2BCFE9C06A7']

In [ ]:
cols = X_test.columns
cols

In [ ]:
rli = []
for i in tqdm(range(len(cli[:5]))):
    temp = filtered_df[filtered_df['magic_key'] == cli[i]][cols]
    rli.append( xgb_reg.predict(temp))

rli

In [ ]:
for index,row in sorted_df[:10000].iterrows():
    if row['magic_key']==cli[4]:
        print(row['meat'],'\n')

In [ ]:
rli = []
for i in tqdm(range(len(cli))):
    temp = filtered_df[filtered_df['magic_key'] == cli[i]][cols]
    if(cli[i] == '290D33249B7'):
        rli.append(2.1601956)
    else:
        rli.append(catboost_reg.predict(temp)[0])

In [ ]:
rli[:5]

In [ ]:
np.mean(rli), np.median(rli)

In [ ]:
len(rli), len(cli)

In [ ]:
rli2 = []
for i in tqdm(range(len(cli))):
    temp = filtered_df[filtered_df['magic_key'] == cli[i]]['meat'].tolist()
#     print(temp)
    if(cli[i] == '290D33249B7'):
        rli2.append(0)
    else:
        rli2.append(temp[0])

In [ ]:
rli2[:5]

In [ ]:
purchase.head(2)

In [ ]:
filtered_rows = purchase[purchase['magic_key'] == '290D33249B7']

# Print the filtered rows
print(filtered_rows)

In [ ]:
filtered_rows = prb3[prb3['MAGIC_KEY'] == '290D33249B7']

# Print the filtered rows
print(filtered_rows)

In [ ]:
sorted_df.magic_key.nunique()

In [ ]:
prb3.MAGIC_KEY.nunique()

In [ ]:
'290D33249B7' in prb3.MAGIC_KEY.tolist()

In [ ]:
sub.head()

In [ ]:
prb3['MEAT'] = rli
prb3['MEAT'] = prb3['MEAT'].abs()

In [ ]:
prb3.at[1829, 'MEAT'] = np.mean(rli)
prb3['MEAT'] = prb3['MEAT'].round(1)

prb3['MEAT'][1829]

In [ ]:
prb3

In [ ]:
filtered_rows = prb3[prb3['MAGIC_KEY'] == '290D33249B7']

# Print the filtered rows
print(filtered_rows)

In [ ]:
ri = np.random.randint(0, 1000)

print('index', ri)

filtered_rows = sorted_df[sorted_df['magic_key'] == cli[ri]]

# Print the desired columns of the filtered rows
print(filtered_rows[['meat', 'milk', 'date']])

print('pred', rli[ri].round(1))


In [ ]:
negative_values_exist = (prb3['MEAT'] < 0).any()
negative_values_exist

In [ ]:
prb3.drop(['magic_key'], axis=1, inplace=True)

In [ ]:
prb3.head(3)

In [ ]:
li1 = prb3.MEAT.tolist()
li2 = prb3.MEAT2.tolist()

li1[:4], li2[:4]

In [ ]:
np.array_equal(li1, li2)

In [ ]:
for i, (elem1, elem2) in enumerate(zip(li1, li2)):
    if elem1 != elem2:
        print(f"Values at index {i} do not match: {elem1} != {elem2}")

In [ ]:
prb3.head(2)

In [ ]:
prbx = prb3[['MAGIC_KEY', 'MEAT2']].copy()
prbx.rename(columns={'MEAT2': 'MEAT'}, inplace=True)

prbx

In [ ]:
prb3.shape

In [ ]:
prbx.to_csv('submission810.csv', index=False)

In [ ]:
subss = pd.read_csv('/kaggle/working/submission810.csv')
subss

In [ ]:
prb3

## Do it Later

In [ ]:

# Define all your models in a list with their respective hyperparameters
models = [
    {'model': LinearRegression(), 'params': {'fit_intercept': [True, False]}},
    {'model': RandomForestRegressor(), 'params': {'n_estimators': [100, 200], 'max_depth': [5, 10]}},
#     {'model': SVR(), 'params': {'kernel': ['linear', 'rbf'], 'C': [0.1, 1.0]}},
#     {'model': Lasso(), 'params': {'alpha': [0.01, 0.1]}},
#     {'model': Ridge(), 'params': {'alpha': [0.01, 0.1]}},
]

# Loop through each model, perform grid search, and evaluate using RMSE and R-squared
for model_def in models:
    model_name = model_def['model'].__class__.__name__
    model = model_def['model']
    params = model_def['params']

    # Perform grid search to find the best parameters
    grid_search = GridSearchCV(model, params, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    # Make predictions and evaluate
    y_pred = best_model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)  # Calculate RMSE
    r2 = r2_score(y_test, y_pred)  # Calculate R-squared

    # Print results for the best model configuration
    print(f"Model: {model_name} - Best Parameters: {grid_search.best_params_}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R-squared: {r2:.4f}")
    print("-" * 30)  #


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_error_difference(error_matrices, model_names):
    """
    Plot the difference between error matrices of different models.

    Parameters:
        error_matrices (list of arrays): List containing the error matrices of different models.
        model_names (list of str): List containing the names of the models.

    Returns:
        None
    """
    num_models = len(error_matrices)

    # Calculate the pairwise difference between error matrices
    pairwise_diff = []
    for i in range(num_models):
        for j in range(i + 1, num_models):
            diff = error_matrices[i] - error_matrices[j]
            pairwise_diff.append(diff)

    num_diff_matrices = len(pairwise_diff)

    # Plotting
    fig, axes = plt.subplots(nrows=1, ncols=num_diff_matrices, figsize=(15, 5))

    for i, diff_matrix in enumerate(pairwise_diff):
        ax = axes[i] if num_diff_matrices > 1 else axes
        im = ax.imshow(diff_matrix, cmap='coolwarm', aspect='auto')
        ax.set_title(f'Difference between {model_names[i]} and {model_names[i+1]}')
        ax.set_xlabel('Actual')
        ax.set_ylabel('Predicted')

    # Colorbar
    cbar = fig.colorbar(im, ax=axes.ravel().tolist())
    cbar.set_label('Difference')

    plt.tight_layout()
    plt.show()
